# Egy állatszakértő rendszer megvalósítása

Példa az [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) tananyagból.

Ebben a mintában egy egyszerű, tudásalapú rendszert valósítunk meg egy állat meghatározására néhány fizikai jellemző alapján. A rendszert a következő AND-OR fa ábrázolja (ez a fa egy része, könnyen hozzáadhatunk több szabályt): 

![](../../../../../../translated_images/hu/AND-OR-Tree.5592d2c70187f283.webp)


## Saját szakértői rendszer shell visszafelé való következtetéssel

Próbáljunk meg definiálni egy egyszerű nyelvet tudásreprezentációhoz, amely alapul veszi a produkciós szabályokat. Python osztályokat fogunk kulcsszavakként használni a szabályok definiálásához. Lényegében 3 fajta osztály lenne:
* Az `Ask` egy kérdést reprezentál, amit a felhasználónak fel kell tenni. Tartalmazza a lehetséges válaszok halmazát.
* Az `If` egy szabályt reprezentál, és ez csak egy szintaktikai cukorka a szabály tartalmának tárolására.
* Az `AND`/`OR` osztályok az AND/OR ágakat reprezentálják a fában. Csak az argumentumok listáját tárolják belül. A kód egyszerűsítése érdekében minden funkcionalitás a szülő osztályban, a `Content`-ben van definiálva.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Rendszerünkben a munkamemória tartalmazná a **tények** listáját **attribútum-érték párokként**. A tudásbázist egy nagy szótárként lehet definiálni, amely az akciókat (új tényeket, amelyeket be kell illeszteni a munkamemóriába) feltételekhez rendeli, amelyeket ÉS-VAGY kifejezésekkel fejeznek ki. Továbbá, néhány tényet meg lehet `Ask`-elni.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

A visszafelé történő következtetés végrehajtásához definiálni fogjuk a `Knowledgebase` osztályt. Ez fogja tartalmazni:
* Működő `memory` - egy szótár, amely attribútumokat értékekhez rendel
* Tudásbázis `rules` az előzőekben meghatározott formátumban

Két fő metódus:
* `get`, amely egy attribútum értékét szerzi meg, szükség esetén következtetést végezve. Például a `get('color')` lekéri a szín mező értékét (ha kell, megkérdezi, majd eltárolja az értéket későbbi használathoz a munkamemóriában). Ha a `get('color:blue')` kérjük, megkérdezi a színt, majd az alapján `y`/`n` értékkel tér vissza.
* `eval`, amely maga a tényleges következtetést végzi, azaz végigjárja az ÉS/VAGY fát, kiértékeli az al-célokat, stb.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Most határozzuk meg állatismereti adatbázisunkat, és végezzük el a konzultációt. Vegye figyelembe, hogy ez a hívás kérdéseket fog feltenni. Igen-nem kérdésekre `y`/`n` gépelésével válaszolhat, vagy hosszabb, többválasztásos kérdések esetén szám (0..N) megadásával.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Experta használata előrehaladó következtetéshez

A következő példában megpróbáljuk megvalósítani az előrehaladó következtetést a tudásreprezentációs könyvtárak egyike, az [Experta](https://github.com/nilp0inter/experta) segítségével. Az **Experta** egy Pythonban használható könyvtár előrehaladó következtetési rendszerek létrehozásához, amely a klasszikus, régi rendszerhez, a [CLIPS](http://www.clipsrules.net/index.html) nevűhöz hasonlóan lett kialakítva.

Az előrehaladó láncolást mi magunk is megvalósíthattuk volna gond nélkül, de a naiv megvalósítások általában nem túl hatékonyak. A szabályok hatékonyabb egyeztetése érdekében egy speciális, [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) nevű algoritmust használnak.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Rendszerünket úgy definiáljuk, hogy egy `KnowledgeEngine` alosztály legyen. Minden szabályt külön funkcióban definiálunk `@Rule` annotációval, amely meghatározza, mikor kell a szabálynak lefutnia. A szabályon belül új tényeket adhatunk hozzá a `declare` függvény használatával, és ezeknek a tényeknek a hozzáadása további szabályok meghívását eredményezi a előre irányuló következtető motor által.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Miután meghatároztuk a tudásbázist, néhány kezdeti ténnyel feltöltjük a munkamemóriánkat, majd meghívjuk a `run()` metódust az következtetés elvégzéséhez. Az eredményként látható, hogy új következtetett tények kerülnek hozzáadásra a munkamemóriához, beleértve a végső tényt az állatról (ha helyesen állítottuk be az összes kezdeti tényt).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Nyilatkozat**:
Ezt a dokumentumot az AI fordító szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) használatával fordítottuk. Bár az pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum a saját nyelvén tekintendő hivatalos forrásnak. Fontos információk esetén profi emberi fordítást javaslunk. Nem vállalunk felelősséget az ebből az fordításból eredő félreértésekért vagy félreértelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
